<a href="https://colab.research.google.com/github/Cutie-tee/nlp_project/blob/main/nlp_textclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the Data

In [23]:
import pandas as pd
#Load the dataset
data =pd.read_csv('/content/training_data_lowercase.csv')
#print(data.head())
print(data.columns)
#print(data.head())

# column names
column_names = data.columns.tolist()
print("Column names:", column_names)

Index(['0\tdonald trump sends out embarrassing new year‚s eve message; this is disturbing'], dtype='object')
Column names: ['0\tdonald trump sends out embarrassing new year‚s eve message; this is disturbing']


In [25]:
# Reload the CSV with tab delimiter
data = pd.read_csv('/content/training_data_lowercase.csv', delimiter='\t')

# Rename the columns for clarity
data.columns = ['label', 'text']
# Display the first few rows to confirm
print(data.head())


   label                                               text
0      0  drunk bragging trump staffer started russian c...
1      0  sheriff david clarke becomes an internet joke ...
2      0  trump is so obsessed he even has obama‚s name ...
3      0  pope francis just called out donald trump duri...
4      0  racist alabama cops brutalize black boy while ...


Data Preprocessing ( Cleaning& Tokenisation)

In [30]:
#Clean the text to convert to lowercase, remove punctuation, whitespace , stop words, toeksnising andlemmatising

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Define stop words and preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and remove stopwords
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column
data['cleaned_text'] = data['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Converting Test to Numerical Features

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['cleaned_text'])
y = data['label']  # Assuming 'label' column is already numeric; if not, convert it with LabelEncoder


 Vectorize Text Data with TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for efficiency
X = vectorizer.fit_transform(data['cleaned_text'])
y = data['label']  # Assuming 'label' is the target variable


Train-Test Split

In [32]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
